# Предобработка данных Ростуризма для собственного дашборда – Внутренний турпоток

Проблематика работы:
- Выгрузка данных от Ростуризма неудобная, содержит накопленные квартальные значения по годам, вместо значений по кварталам
- Нужно предобработать данные с отчетов Ростуризма: https://rosstat.gov.ru/statistics/turizm для построения собственного дашборда для мониторинга основных показателей индустрии


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from numpy import median


pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)



In [2]:
stays_22 = pd.read_excel ('Дашборд по туризму/Turpotok_05-2025.xlsx', sheet_name='2.1.2022', skiprows=5)
stays_23 = pd.read_excel ('Дашборд по туризму/Turpotok_05-2025.xlsx', sheet_name='2.1.2023', skiprows=5)
stays_24 = pd.read_excel ('Дашборд по туризму/Turpotok_05-2025.xlsx', sheet_name='2.1.2024', skiprows=5)

nights_23 = pd.read_excel ('Дашборд по туризму/Turpotok_05-2025.xlsx', sheet_name='2.2.2023', skiprows=5)
nights_24 = pd.read_excel ('Дашборд по туризму/Turpotok_05-2025.xlsx', sheet_name='2.2.2024', skiprows=5)

monthly_st_22 = pd.read_excel ('Дашборд по туризму/Turpotok_05-2025.xlsx', sheet_name='3.2022', skiprows=5)
monthly_st_23 = pd.read_excel ('Дашборд по туризму/Turpotok_05-2025.xlsx', sheet_name='3.2023', skiprows=5)
monthly_st_24 = pd.read_excel ('Дашборд по туризму/Turpotok_05-2025.xlsx', sheet_name='3.2024', skiprows=5)
monthly_st_25 = pd.read_excel ('Дашборд по туризму/Turpotok_05-2025.xlsx', sheet_name='3.2025', skiprows=5)

In [3]:
nights_22 = nights_23[0:0]

In [4]:
stays_list = [stays_22,stays_23,stays_24]
nights_list = [nights_22,nights_23,nights_24]
monthly_list = [monthly_st_22,monthly_st_23,monthly_st_24,monthly_st_25]

In [5]:
data = pd.DataFrame(columns=['date','region','stays','nights'])

In [6]:
data

,date,region,stays,nights


In [7]:
stays_23

,Unnamed: 0,январь-март,январь-июнь,январь-сентябрь,январь-декабрь
0,Российская Федерация,26456109.00,66297745.00,121697302.00,153051341.00
1,Центральный федеральный округ,9114447.00,23055750.00,39178686.00,50831408.00
2,Белгородская область,158134.00,278586.00,472479.00,584940.00
3,Брянская область,87082.00,201906.00,443211.00,611730.00
4,Владимирская область,213101.00,496071.00,995329.00,1181734.00
...,...,...,...,...,...
92,Магаданская область,17284.00,38821.00,66623.00,86539.00
93,Сахалинская область,97659.00,160149.00,249917.00,307254.00
94,Еврейская автономная область,8014.00,30679.00,43642.00,54035.00
95,Чукотский автономный округ,8793.00,18589.00,57941.00,66008.00


In [8]:
a=2015
pd.to_datetime('01-03-'+str(a))

Timestamp('2015-01-03 00:00:00')

In [9]:
for i in range(len(stays_list)):
    a=2022+i
#    display(a)
    stay = stays_list[i]
    night = nights_list[i]
    stay.columns = stay.columns.str.replace('январь-декабрь1)','январь-декабрь')
    stay.columns = stay.columns.str.replace('январь-сентябрь1)','январь-сентябрь')
    stay.columns = stay.columns.str.replace('январь-июнь1)','январь-июнь')
    stay.columns = stay.columns.str.replace('январь-март1)','январь-март')
    night.columns = night.columns.str.replace('январь-декабрь1)','январь-декабрь')
    night.columns = night.columns.str.replace('январь-сентябрь1)','январь-сентябрь')
    night.columns = night.columns.str.replace('январь-июнь1)','январь-июнь')
    night.columns = night.columns.str.replace('январь-март1)','январь-март')
    
    stay.columns = stay.columns.str.replace('январь-декабрь*','январь-декабрь')
    night.columns = night.columns.str.replace('январь-декабрь*','январь-декабрь')
    
#    print (a, 'stay')
#    display(stay)
#    print (a, 'night')
#    display(night)
    
    year = stay.merge(night, how='outer', on='Unnamed: 0')
    year = year.replace(',','.')

    year['январь-декабрь_x'] = pd.to_numeric(year['январь-декабрь_x'], errors='coerce')
    year['январь-сентябрь_x'] = pd.to_numeric(year['январь-сентябрь_x'], errors='coerce')
    year['январь-июнь_x'] = pd.to_numeric(year['январь-июнь_x'], errors='coerce')
    year['январь-март_x'] = pd.to_numeric(year['январь-март_x'], errors='coerce')
    year['январь-декабрь_y'] = pd.to_numeric(year['январь-декабрь_y'], errors='coerce')
    year['январь-сентябрь_y'] = pd.to_numeric(year['январь-сентябрь_y'], errors='coerce')
    year['январь-сентябрь_y'] = pd.to_numeric(year['январь-сентябрь_y'], errors='coerce')
    year['январь-март_y'] = pd.to_numeric(year['январь-март_y'], errors='coerce')
    
    year['январь-декабрь_x'] = (year['январь-декабрь_x'] - year['январь-сентябрь_x'])
    year['январь-сентябрь_x'] = (year['январь-сентябрь_x'] - year['январь-июнь_x'])
    year['январь-июнь_x'] = (year['январь-июнь_x'] - year['январь-март_x'])
    year['январь-март_x'] = year['январь-март_x']
    year['январь-декабрь_y'] = (year['январь-декабрь_y'] - year['январь-сентябрь_y'])
    year['январь-сентябрь_y'] = (year['январь-сентябрь_y'] - year['январь-июнь_y'])
    year['январь-июнь_y'] = (year['январь-июнь_y'] - year['январь-март_y'])
    year['январь-март_y'] = year['январь-март_y']

#    print (a, 'year')
#    display(year)

    q1= year[['Unnamed: 0','январь-март_x','январь-март_y']].rename(columns={'Unnamed: 0':'region','январь-март_x':'stays','январь-март_y':'nights'})
    q1['date'] = pd.to_datetime('30-03-'+str(a),dayfirst=True)
#    print (a, 'q1')
#    display(q1)

    q2= year[['Unnamed: 0','январь-июнь_x','январь-июнь_y']].rename(columns={'Unnamed: 0':'region','январь-июнь_x':'stays','январь-июнь_y':'nights'})
    q2['date'] = pd.to_datetime('30-06-'+str(a),dayfirst=True)
#    print (a, 'q2')
#    display(q2)

    q3= year[['Unnamed: 0','январь-сентябрь_x','январь-сентябрь_y']].rename(columns={'Unnamed: 0':'region','январь-сентябрь_x':'stays','январь-сентябрь_y':'nights'})
    q3['date'] = pd.to_datetime('30-09-'+str(a),dayfirst=True)
#    print (a, 'q3')
#    display(q3)

    q4= year[['Unnamed: 0','январь-декабрь_x','январь-декабрь_y']].rename(columns={'Unnamed: 0':'region','январь-декабрь_x':'stays','январь-декабрь_y':'nights'})
    q4['date'] = pd.to_datetime('30-12-'+str(a),dayfirst=True)
#    print (a, 'q4')
#    display(q4)

    data = pd.concat([data,q1,q2,q3,q4])

C:\Users\user\AppData\Local\Temp\ipykernel_4024\951424477.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data,q1,q2,q3,q4])


In [10]:
#data


In [11]:
#data.info()

In [12]:
#data['region'].value_counts()

In [13]:
data = data[data['region'] != '* Без учета статистической информации по Донецкой Народной Республике (ДНР), Луганской Народной Республике (ЛНР), Запорожской и Херсонской областям\n** Без учета временно эвакуированных граждан']

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1152 entries, 1 to 96
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1152 non-null   datetime64[ns]
 1   region  1152 non-null   object        
 2   stays   1152 non-null   float64       
 3   nights  768 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 45.0+ KB


In [15]:
data.to_excel('Дашборд по туризму/tourism_inner_quarter.xlsx', index=False)

In [16]:
data_month = pd.DataFrame(columns=['date','region','stays'])
data_month

,date,region,stays


In [17]:
for i in range(len(monthly_list)):
    a=2022+i
#    display(a)

    
    month = monthly_list[i]
    month.columns = month.columns.str.replace('*','')
    
    month['декабрь'] = (month['январь-декабрь'] - month['январь-ноябрь'])
    month['ноябрь'] = (month['январь-ноябрь'] - month['январь-октябрь'])
    month['октябрь'] = (month['январь-октябрь'] - month['январь-сентябрь'])
    month['сентябрь'] = (month['январь-сентябрь'] - month['январь-август'])
    month['август'] = (month['январь-август'] - month['январь-июль'])
    month['июль'] = (month['январь-июль'] - month['январь-июнь'])
    month['июнь'] = (month['январь-июнь'] - month['январь-май'])
    month['май'] = (month['январь-май'] - month['январь-апрель'])
    month['апрель'] = (month['январь-апрель'] - month['январь-март'])
    month['март'] = (month['январь-март'] - month['январь-февраль'])
    month['февраль'] = (month['январь-февраль'] - month['январь'])

#    print (a, 'year')
#    display(month)

    m1= month[['Unnamed: 0','январь']].rename(columns={'Unnamed: 0':'region','январь':'stays'})
    m1['date'] = pd.to_datetime('30-01-'+str(a),dayfirst=True)
#    print (a, 'm1')
#    display(m1)

    m2= month[['Unnamed: 0','февраль']].rename(columns={'Unnamed: 0':'region','февраль':'stays'})
    m2['date'] = pd.to_datetime('28-02-'+str(a),dayfirst=True)
#    print (a, 'm2')
#    display(m2)

    m3= month[['Unnamed: 0','март']].rename(columns={'Unnamed: 0':'region','март':'stays'})
    m3['date'] = pd.to_datetime('30-03-'+str(a),dayfirst=True)
#    print (a, 'm3')
#    display(m3)

    m4= month[['Unnamed: 0','апрель']].rename(columns={'Unnamed: 0':'region','апрель':'stays'})
    m4['date'] = pd.to_datetime('30-04-'+str(a),dayfirst=True)
#    print (a, 'm4')
#    display(m4)

    m5= month[['Unnamed: 0','май']].rename(columns={'Unnamed: 0':'region','май':'stays'})
    m5['date'] = pd.to_datetime('30-05-'+str(a),dayfirst=True)
#    print (a, 'm5')
#    display(m5)

    m6= month[['Unnamed: 0','июнь']].rename(columns={'Unnamed: 0':'region','июнь':'stays'})
    m6['date'] = pd.to_datetime('30-06-'+str(a),dayfirst=True)
#    print (a, 'm6')
#    display(m6)

    m7= month[['Unnamed: 0','июль']].rename(columns={'Unnamed: 0':'region','июль':'stays'})
    m7['date'] = pd.to_datetime('30-07-'+str(a),dayfirst=True)
#    print (a, 'm7')
#    display(m7)

    m8= month[['Unnamed: 0','август']].rename(columns={'Unnamed: 0':'region','август':'stays'})
    m8['date'] = pd.to_datetime('30-08-'+str(a),dayfirst=True)
#    print (a, 'm8')
#    display(m8)

    m9= month[['Unnamed: 0','сентябрь']].rename(columns={'Unnamed: 0':'region','сентябрь':'stays'})
    m9['date'] = pd.to_datetime('30-09-'+str(a),dayfirst=True)
#    print (a, 'm9')
#    display(m9)

    m10= month[['Unnamed: 0','октябрь']].rename(columns={'Unnamed: 0':'region','октябрь':'stays'})
    m10['date'] = pd.to_datetime('30-10-'+str(a),dayfirst=True)
#    print (a, 'm10')
#    display(m10)

    m11= month[['Unnamed: 0','ноябрь']].rename(columns={'Unnamed: 0':'region','ноябрь':'stays'})
    m11['date'] = pd.to_datetime('30-11-'+str(a),dayfirst=True)
#    print (a, 'm11')
#    display(m11)

    m12= month[['Unnamed: 0','декабрь']].rename(columns={'Unnamed: 0':'region','декабрь':'stays'})
    m12['date'] = pd.to_datetime('30-12-'+str(a),dayfirst=True)
#    print (a, 'm12')
#    display(m12)

    data_month = pd.concat([data_month,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12])

C:\Users\user\AppData\Local\Temp\ipykernel_4024\2742012136.py:84: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_month = pd.concat([data_month,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12])


In [18]:
data_month['region'].value_counts()


region
Российская Федерация                                                                                                                                                                                 48
Ставропольский край                                                                                                                                                                                  48
Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)                                                                                             48
Ямало-Ненецкий автономный округ                                                                                                                                                                      48
Ханты-Мансийский автономный округ - Югра                                                                                                                                                         

In [19]:
data_month.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4656 entries, 0 to 96
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4656 non-null   datetime64[ns]
 1   region  4656 non-null   object        
 2   stays   3936 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 145.5+ KB


In [20]:
data_month = data_month[data_month['region'] != '* Без учета статистической информации по Донецкой Народной Республике (ДНР), Луганской Народной Республике (ЛНР), Запорожской и Херсонской областям\n** Без учета временно эвакуированных граждан']

In [21]:
data_month.to_excel('Дашборд по туризму/tourism_inner_month.xlsx', index=False)